# Simple FRAP script

This script analyses all Images in a Dataset, measuring the intensity in a named Channel within a ROI. The intensity is plotted over time.

### Import packages

In [19]:
from PIL import Image
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from numpy import array, int8
from skimage import feature
from scipy.ndimage import convolve
from scipy import misc
from scipy import ndimage
from omero.model import EllipseI

### Create a connection to the OMERO Server

In [20]:
from omero.gateway import BlitzGateway

from getpass import getpass

HOST = 'merge-ci-devspace.openmicroscopy.org'
PORT = 4064
conn = BlitzGateway(raw_input("Username: "), getpass("OMERO Password: "), host=HOST, port=PORT)
conn.connect()

In [28]:
dataset_id = 5220
dataset = conn.getObject("Dataset", dataset_id)
images = []
for image in dataset.listChildren():
    print image.id, image.name
    images.append(image)
images = images[:2]

In [29]:
def getEllipse(image):
    roi_service = conn.getRoiService()
    result = roi_service.findByImage(image.getId(), None)
    # Simply return any Ellipse we find...
    shape_id = None
    for roi in result.rois:
        print "ROI:  ID:", roi.getId().getValue()
        for s in roi.copyShapes():
            if type(s) == EllipseI:
                shape_id = s.id.val
    print "Shape:", shape_id
    return shape_id

In [30]:
def getMeanIntensities(image, shape_id):
    # Get pixel intensities for first Channel
    roi_service = conn.getRoiService()
    the_c = 0
    the_z = 0
    size_t = image.getSizeT()
    print 'SizeT', size_t
    meanvalues = []
    for t in range(size_t):
        stats = roi_service.getShapeStatsRestricted([shape_id],
                                                    the_z, t, [the_c])
        meanvalues.append(stats[0].mean[the_c])

    print meanvalues

In [32]:
for image in images:
    shape_id = getEllipse(image)
    if shape_id is None:
        continue
    intensities = getMeanIntensities(image, shape_id)

<br>
#### When you are completely finished running the notebook close the connection to OMERO server

In [ ]:
conn.close()